## Maestría en Analítica de datos
## Materia Big Data
## Clase 4

*   Nombre: José Luis Ariza
*   Código: 80125197


# Taller básico sobre MONGOdb
----


1.   Instalar librerías
2.   descargar e instalar un Sevidor propio de MongoDb
3.   Crear una Bd
4.   Crear colecciones
5.   descargar .zip (csv) cargarlos en una Colección



# 1.Instalar librerías

In [1]:
!apt update
!apt install -y python3-pip
!pip3 install pymongo        #libreria especial

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,571 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,286 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,081 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers

# 2.Descargar e instalar un servidor propio de MongoDB

In [2]:
!sudo apt-get install -y gnupg curl
!curl -fsSL https://pgp.mongodb.com/server-6.0.asc | sudo gpg -o /usr/share/keyrings/mongodb-server-6.0.gpg --dearmor
!echo "deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-6.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-6.0.list

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
curl is already the newest version (7.81.0-1ubuntu1.21).
gnupg is already the newest version (2.2.27-3ubuntu2.4).
gnupg set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 58 not upgraded.
deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-6.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 multiverse


In [3]:
!sudo apt-get update
!sudo apt-get install -y mongodb-org

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 InRelease [4,009 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:10 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0/multiverse arm64 Packages [107 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0/multiverse amd64 Packages [110 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:14 https://ppa.launch

## 2.1 instalar mongoDB en carpeta de google drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
db_path = '/content/drive/MyDrive/Ucentral/2025S2/BigData/MongoDB/'
os.makedirs(db_path, exist_ok=True)
!sudo chmod 777 $db_path  #permisos especiales de escritura sobre esa carpeta

In [6]:
#modifico las variables de entorno del SO para que reconozca el servidor
!sudo systemctl stop mongod
!sudo mongod --dbpath $db_path --fork --logpath /var/log/mongodb/mongod.log

System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
about to fork child process, waiting until server is ready for connections.
forked process: 4035
child process started successfully, parent exiting


# 3. inicializar nuestro servidor mongoDB

In [7]:
from pymongo import MongoClient
client = MongoClient('localhost',27017)

## 3.1 crear una base de datos

In [8]:
db = client['estudiantes_Ucentral']

In [9]:
# crear colecciones
coleccion_profes = db['profesores']
coleccion_cursos = db['cursos']

## 3.2 funciones DML (insertar, borrar, actualizar)

In [10]:
def buscar_documentos(db, coleccion_nombre, filtro={}, limite=10):
  if db is not None:
      coleccion = db[coleccion_nombre]
      resultados = coleccion.find(filtro).limit(limite)
      for doc in resultados:
          print(doc)
  else:
      print("No hay conexión a la base de datos.")

def insertar_documento(db, coleccion_nombre, documento):
      if db is not None:
          coleccion = db[coleccion_nombre]
          resultado = coleccion.insert_one(documento)
          print(f"Documento insertado con ID: {resultado.inserted_id}")
      else:
          print("No hay conexión a la base de datos.")

def actualizar_un_documento(db, coleccion_nombre, filtro, actualizacion):
        if db is not None:
            coleccion = db[coleccion_nombre]
            resultado = coleccion.update_one(filtro, {'$set': actualizacion})
            if resultado.modified_count > 0:
                print(f"Documentos modificados: {resultado.modified_count}")
            else:
                print("No se encontraron documentos para modificar.")
        else:
            print("No hay conexión a la base de datos.")
def actualizar_varios_documentos(db, coleccion_nombre, filtro, actualizacion):
        if db is not None:
            coleccion = db[coleccion_nombre]
            resultado = coleccion.update_many(filtro, {'$set': actualizacion})
            if resultado.modified_count > 0:
                print(f"Documentos modificados: {resultado.modified_count}")
            else:
                print("No se encontraron documentos para modificar.")
        else:
            print("No hay conexión a la base de datos.")
def eliminar_varios_documentos(db, coleccion_nombre, filtro):
        if db is not None:
            coleccion = db[coleccion_nombre]
            resultado = coleccion.delete_many(filtro)
            if resultado.deleted_count > 0:
                print(f"Documentos eliminados: {resultado.deleted_count}")
            else:
                print("No se encontraron documentos para eliminar.")
        else:
            print("No hay conexión a la base de datos.")

In [11]:
documentoProfe1={"nombre":"luisfdo","apellidos":"castellanos guarin"}
documentoProfe2={"nombre":"pepito","apellidos":"perez","fechaNacimiento":"1980-05-05"}
insertar_documento(db,'profesores',documentoProfe1)
insertar_documento(db,'profesores',documentoProfe2)

Documento insertado con ID: 69328723bf520d8724f1635f
Documento insertado con ID: 69328723bf520d8724f16360


In [12]:
buscar_documentos(db,'profesores')

{'_id': ObjectId('69328723bf520d8724f1635f'), 'nombre': 'luisfdo', 'apellidos': 'castellanos guarin'}
{'_id': ObjectId('69328723bf520d8724f16360'), 'nombre': 'pepito', 'apellidos': 'perez', 'fechaNacimiento': '1980-05-05'}


In [13]:
actualizar_varios_documentos(db,'profesores',{"nombre":"pepito"},{"nombre":"Pedro Patricio"})

Documentos modificados: 1


In [14]:
eliminar_varios_documentos(db,'profesores',{"apellidos":"perez"})

Documentos eliminados: 1


In [ ]:
client.close()

#4 descargar y descomprimir zip

In [15]:
def descargar_y_descomprimir_zip(url, carpeta_destino, tipoArchivo=''):
  import os
  import io
  import zipfile
  import requests
  os.makedirs(carpeta_destino, exist_ok=True)  #cree la carpeta sino existe
  response = requests.get(url)
  zip_file = zipfile.ZipFile(io.BytesIO(response.content))
  if (tipoArchivo == ''):
    zip_file.extractall(carpeta_destino) #exportar .zip a la carpeta
  else:
    for nombre_archivo in zip_file.namelist():
      if nombre_archivo.endswith(tipoArchivo):
        zip_file.extract(nombre_archivo, carpeta_destino)


In [16]:
carpeta_destino ="/content/drive/MyDrive/Ucentral/2025S2/BigData/Clases/DataBase/zip_files/"
url             ="https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip"
descargar_y_descomprimir_zip(url,carpeta_destino)

# 5.Procesar data descargada y cargar a una coleccion

In [17]:
import re
def procesar_txt_diccionario_separados_por_espacio(data_diccionario):
  lineas=data_diccionario.strip().split('\n')
  #---listar
  documentos=[]
  for linea in lineas:
    match = re.match('^\d+\s+(.+?)\s*-\s*(.+)$',linea)
    if match:
      clave= match.group(1)
      valor= match.group(2)
      documentos.append({"clave":clave, "valor":valor})
  return documentos

<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-2554699229.py:7: SyntaxWarning: invalid escape sequence '\d'
  match = re.match('^\d+\s+(.+?)\s*-\s*(.+)$',linea)


In [18]:
with open(carpeta_destino+'student.txt', 'r') as archivo:
    data_diccionario = archivo.read()
documentos=procesar_txt_diccionario_separados_por_espacio(data_diccionario)
print(documentos)

[{'clave': 'school', 'valor': 'student\'s school (binary: "GP" - Gabriel Pereira or "MS" - Mousinho da Silveira)'}, {'clave': 'sex', 'valor': 'student\'s sex (binary: "F" - female or "M" - male)'}, {'clave': 'age', 'valor': "student's age (numeric: from 15 to 22)"}, {'clave': 'address', 'valor': 'student\'s home address type (binary: "U" - urban or "R" - rural)'}, {'clave': 'famsize', 'valor': 'family size (binary: "LE3" - less or equal to 3 or "GT3" - greater than 3)'}, {'clave': 'Pstatus', 'valor': 'parent\'s cohabitation status (binary: "T" - living together or "A" - apart)'}, {'clave': 'Medu', 'valor': "mother's education (numeric: 0 - none,  1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)"}, {'clave': 'Fedu', 'valor': "father's education (numeric: 0 - none,  1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)"}, {'clave': 'Mjob', 'valor': 'mother\'s job (nominal: "teache

In [19]:
db = client['estudiantes_Ucentral']
coleccion_estudiantes = db['estudiantes']
for documento in documentos:
  insertar_documento(db,'estudiantes',documento)

Documento insertado con ID: 693287a1bf520d8724f16361
Documento insertado con ID: 693287a1bf520d8724f16362
Documento insertado con ID: 693287a1bf520d8724f16363
Documento insertado con ID: 693287a1bf520d8724f16364
Documento insertado con ID: 693287a1bf520d8724f16365
Documento insertado con ID: 693287a1bf520d8724f16366
Documento insertado con ID: 693287a1bf520d8724f16367
Documento insertado con ID: 693287a1bf520d8724f16368
Documento insertado con ID: 693287a1bf520d8724f16369
Documento insertado con ID: 693287a1bf520d8724f1636a
Documento insertado con ID: 693287a1bf520d8724f1636b
Documento insertado con ID: 693287a1bf520d8724f1636c
Documento insertado con ID: 693287a1bf520d8724f1636d
Documento insertado con ID: 693287a1bf520d8724f1636e
Documento insertado con ID: 693287a1bf520d8724f1636f
Documento insertado con ID: 693287a1bf520d8724f16370
Documento insertado con ID: 693287a1bf520d8724f16371
Documento insertado con ID: 693287a1bf520d8724f16372
Documento insertado con ID: 693287a1bf520d8724

In [20]:
buscar_documentos(db,'estudiantes')

{'_id': ObjectId('693287a1bf520d8724f16361'), 'clave': 'school', 'valor': 'student\'s school (binary: "GP" - Gabriel Pereira or "MS" - Mousinho da Silveira)'}
{'_id': ObjectId('693287a1bf520d8724f16362'), 'clave': 'sex', 'valor': 'student\'s sex (binary: "F" - female or "M" - male)'}
{'_id': ObjectId('693287a1bf520d8724f16363'), 'clave': 'age', 'valor': "student's age (numeric: from 15 to 22)"}
{'_id': ObjectId('693287a1bf520d8724f16364'), 'clave': 'address', 'valor': 'student\'s home address type (binary: "U" - urban or "R" - rural)'}
{'_id': ObjectId('693287a1bf520d8724f16365'), 'clave': 'famsize', 'valor': 'family size (binary: "LE3" - less or equal to 3 or "GT3" - greater than 3)'}
{'_id': ObjectId('693287a1bf520d8724f16366'), 'clave': 'Pstatus', 'valor': 'parent\'s cohabitation status (binary: "T" - living together or "A" - apart)'}
{'_id': ObjectId('693287a1bf520d8724f16367'), 'clave': 'Medu', 'valor': "mother's education (numeric: 0 - none,  1 - primary education (4th grade), 2 